<a href="https://colab.research.google.com/github/ChrisGutknecht/Google-Search-Console-bulk-query-to-GBQ/blob/master/Import_Logfiles_ToBigQuery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. Access Logfile data with credentials
- See: https://jira.bergzeit.de/browse/BWEB-1866
- Directory: https://admin.bergzeit.de/googlebot-logshare/
- Credentials

> - benutzer: googlebot-logs
> - password: |&]H;.S;K%#ymTn0

- Comments
> Die Dateien sind im CSV-Format, Trenner ist ; und die einzelnen Werte sind mit " " umschlossen. Die Dateien stammen (ab dem 29.11.) immer von 23:59, was zur Folge hat, dass jeweils eine Minute dem Folgetag zugerechnet wird. Die IPs sind alle per PTR + A-Record geprüft und stammen somit NUR von Google.

- Docs: https://2.python-requests.org/en/v1.1.0/user/authentication/

- Example File: https://admin.bergzeit.de/googlebot-logshare/2019-11-27.tgz
- Use time package to get current date

In [0]:
import requests 
import pandas as pd
import os
import json
import csv
import io
import tarfile
import time
from dateutil.parser import parse
from datetime import datetime, timedelta
from google.cloud import bigquery

############# CONFIG START #############

# BigQuery constants
dataset_id = 'CrawlLogfileData'
table_name_current = 'DailyCrawlData_All'
project_id = 'bergzeit'

# Logfile Server HTTP Auth credentials
user = 'googlebot-logs'
key = '|&]H;.S;K%#ymTn0'

############# CONFIG END ###############


## In Cloud Functions, set request as the argument >> def runLogfileDataSave(request):
def runLogfileDataSave(): # (request)
    logfileFetcher = LogfileFetcher(user, key)
    dateYesterday = datetime.strftime(datetime.now() - timedelta(2), '%Y-%m-%d')

    logfileData = logfileFetcher.getDataByDate(dateYesterday)
    saveLogfilesToDb(logfileData)


###################################


# Service to fetch logfile crawl content
class LogfileFetcher:
    def __init__(self, user, key):
        self.user = user
        self.key = key
        
    def getDataByDate(self, date):
        url = 'https://admin.bergzeit.de/googlebot-logshare/' + str(date) + '.tgz'
        response = requests.get(url, auth=(self.user, self.key), stream=True)

        if response.status_code == 200:
            with open('/content/file.tar.gz', 'wb') as file:
                file.write(response.raw.read())

        # Unzip folder and extract content
        tar = tarfile.open("/content/file.tar.gz", "r:gz")
        logFileContent = []
        headerRow = ['Remote IP', 'Request Time', 'User Agent', 'Request', 'Status Code', 'Referer', '', 'Domain']
        logFileContent.append(headerRow)

        for tarinfo in tar:
            domain = str(tarinfo.name).split('-accesslog')[0].split('googlebot-')[1].split('-')[0]
            csv_file = io.StringIO(tar.extractfile(tarinfo).read().decode('ascii'))
            iterRows = iter(csv.reader(csv_file, delimiter=';'))
            next(iterRows)

            for row in iterRows:
                listRow = row
                # Add domain from filename
                listRow.append(domain)
                logFileContent.append(listRow)

        tar.extractall()
        tar.close()

        # prepare dataframe
        df = pd.DataFrame.from_records(logFileContent)
        df.columns = df.iloc[0]
        df = df.drop(df.index[0])
        df.rename(columns={
            'Remote IP': 'BotIpAddress','Request Time': 'RequestTime',
            'User Agent': 'UserAgent', 'Request' : 'RequestUrl', 'Status Code' : 'StatusCode'
            }, inplace=True)
        
        df['Referer'] = df.apply (lambda row: self.createReferer(row) , axis=1)
        df['RequestUrl'] = df.apply (lambda row: self.createUrl(row)  ,axis=1)
        df['RequestTime'] = df.apply (lambda row: self.convertToDateTime(row) , axis=1)
        df['Date'] = df.apply (lambda row: self.getDate(row) , axis=1)

        df = df[['Date', 'Domain', 'RequestTime', 'RequestUrl', 'UserAgent', 'StatusCode', 'Referer', 'BotIpAddress' ]]
        print(df.head())

        #Convert dataframe to list of lists
        return df.values.tolist()   # to_json(orient='records')
        
    def createUrl(self, row):
        return row.Domain + row.RequestUrl.replace('GET ', '').replace(' HTTP/1.1', '')
    
    def createReferer(self, row):
        return row.Domain + row.Referer.replace('GET ', '').replace(' HTTP/1.1', '')

    def convertToDateTime(self, row):
        datetime = parse(row.RequestTime.replace(':',' ',1))
        return  datetime.strftime("%Y-%m-%d %H:%M:%S")
        # return  time.strptime(row.RequestTime, "%d/%b/%y:%H:%M:%S %z").strftime("%Y-%m-%d %H:%M:%S %z")	

    def getDate(self, row):
        datetime = parse(row.RequestTime)
        return  datetime.strftime("%Y-%m-%d")
        # return  time.strptime(row.RequestTime,"%Y-%m-%d %H:%M:%S %z").strftime("%Y-%m-%d")


class BigQueryConnector:
    def __init__(self, dataset_id, table_name, schema):
        self.schema = schema
 
        # Instantiates a client
        self.bigquery_client = bigquery.Client(project=project_id)
 
        # Prepares a reference to the new dataset
        dataset_ref = self.bigquery_client.dataset(dataset_id)
        dataset_ref = bigquery.Dataset(dataset_ref)
        dataset = None
        try:
            dataset = self.bigquery_client.get_dataset(dataset_ref)
        except Exception as e:
            raise e
 
        table_ref = dataset.table(table_name)
        self.table = None
        try:
            self.table = self.bigquery_client.get_table(table_ref)
        except Exception as e:
            raise e
 
    def writeRowsToStorage(self, rows):
        errors = ""
        valueList = tuple(rows) # .values()
        row = [valueList]

        try:
            errors = self.bigquery_client.insert_rows(self.table, row)
        except Exception as e:
            print(e)
 
        if len(errors) > 0:
            print(errors)
 
 
def saveLogfilesToDb(data):
    values = data

    currentSchema_raw = [
        bigquery.SchemaField('Date', 'DATE'),
        bigquery.SchemaField('Domain', 'STRING'),
        bigquery.SchemaField('RequestTime', 'TIMESTAMP'),
        bigquery.SchemaField('RequestUrl', 'STRING'),
        bigquery.SchemaField('UserAgent', 'STRING'),
        bigquery.SchemaField('StatusCode', 'STRING'),
        bigquery.SchemaField('Referer', 'STRING'),
        bigquery.SchemaField('BotIpAddress', 'STRING')
    ]

    bq_connector = BigQueryConnector(dataset_id=dataset_id, table_name=table_name_current, schema=currentSchema_raw)

    for value in values:
        bq_connector.writeRowsToStorage(value)  # json.loads()
    print("Done with data handling...")

runLogfileDataSave()

0        Date  ...  BotIpAddress
1  2019-12-06  ...  66.249.76.91
2  2019-12-06  ...  66.249.76.64
3  2019-12-06  ...  66.249.76.75
4  2019-12-06  ...  66.249.76.64
5  2019-12-06  ...  66.249.76.75

[5 rows x 8 columns]


### Reference: Save to weahter data to BigQuery

- Set schema, set dataset and table in BQ

- See weather data cloud function: https://colab.research.google.com/drive/1ERqJkvnYS-v10eLes1voh2iIl6gwsqBO#scrollTo=eVAIBSG7VKeK



In [0]:

# Current weather: http://api.openweathermap.org/data/2.5/weather?q=Otterfing,de&units=metric&appid=bdbb4a147eeeb8717fc10be6e62cbabe
# Forecast: http://api.openweathermap.org/data/2.5/forecast?q=Otterfing,de&units=metric&appid=bdbb4a147eeeb8717fc10be6e62cbabe
# Cloud Function Project: https://console.cloud.google.com/functions/list?project=adsdataprediction

import requests
import json
from datetime import datetime
from pytz import timezone


#  Reference: https://googleapis.github.io/google-cloud-python/latest/bigquery/generated/google.cloud.bigquery.table.Table.html
# Imports the Google Cloud client library
from google.cloud import bigquery

from google.colab import auth
auth.authenticate_user()
from google.cloud import bigquery

############# CONFIG START #############

api_key_owp = 'bdbb4a147eeeb8717fc10be6e62cbabe'
dataset_id = 'weatherData'
table_name_current = 'weatherData_Current'
table_name_forecast = 'weatherData_forecasted'
project_id = 'yourProjectId'

client = bigquery.Client(project=project_id)

currentSchema_raw = [
    bigquery.SchemaField('date', 'DATE'),
    bigquery.SchemaField('country', 'STRING'),
    bigquery.SchemaField('town_name', 'STRING'),
    bigquery.SchemaField('town_id', 'STRING'),
    bigquery.SchemaField('currentTime', 'DATETIME'),
    bigquery.SchemaField('timeZone', 'STRING'),
    bigquery.SchemaField('timeOfDay', 'STRING'),
    bigquery.SchemaField('weekday_Index', 'STRING'),
    bigquery.SchemaField('weekday_Name', 'STRING'),
    bigquery.SchemaField('temperature', 'FLOAT'),
    bigquery.SchemaField('overallWeather', 'STRING'),
    bigquery.SchemaField('overallWeather_code', 'STRING'),
    bigquery.SchemaField('rained', 'BOOL'),
    bigquery.SchemaField('rainAmount_3hrs_mm', 'STRING'),
    bigquery.SchemaField('humidity', 'INTEGER')
]

towns = ['Hamburg,de', 'Berlin,de', 'Hannover,de','Bremen,de', 'Dresden,de', 'Leipzig,de'
    'Dortmund,de', 'Köln,de', 'Frankfurt,de', 'Nürnberg,de', 'Stuttgart,de', 'München,de', 'Passau,de', 'Konstanz,de',
    'Wien,at', 'Linz,at', 'Graz,at', 'Salzburg,at', 'Klagenfurt,at', 'Liezen,at', 
    'Lienz,at', 'Flachau,at', 'Kufstein, at', 'Innsbruck,at', 'Bregenz,at', 'Feldkirch,at', 'Imst,at', 
    'Lausanne,ch', 'Bern,ch', 'Martigny,ch', 'Basel,ch', 'Zürich,ch', 'Luzern,ch', 'Chur,ch', 
    'Sankt Moritz,ch', 'Sankt Gallen,ch', 'Bellinzona,ch', 'Andermatt,ch'
    ]

 
############# CONFIG END ###############

## In Cloud Functions, set request as the argument >> def runWeatherDataSave(request):
def runWeatherDataSave(/*request*/): 
    weatherFetcher = WeatherFetcher(api_key_owp, towns)
    currentWeatherData = weatherFetcher.getCurrentWeatherData()
    weatherDataWriter(currentWeatherData)

########

class BigQueryConnector:
    def __init__(self, dataset_id, table_name, schema):
        self.schema = schema
 
        # Instantiates a client
        self.bigquery_client = bigquery.Client(project=project_id)
 
        # Prepares a reference to the new dataset
        dataset_ref = self.bigquery_client.dataset(dataset_id)
        dataset_ref = bigquery.Dataset(dataset_ref)
        dataset = None
        try:
            dataset = self.bigquery_client.get_dataset(dataset_ref)
        except Exception as e:
            if dataset is None:
                # Creates the new dataset
                dataset = self.bigquery_client.create_dataset(dataset_ref)
                print('Dataset {} created.'.format(dataset.dataset_id))
            else:
                raise e
 
        table_ref = dataset.table(table_name)
        self.table = None
        try:
            self.table = self.bigquery_client.get_table(table_ref)
        except Exception as e:
            if self.table is None:
                self.table = bigquery.Table(table_ref, schema=self.schema)
                self.table = self.bigquery_client.create_table(self.table)
            else:
                raise e
 
    def writeRowsToStorage(self, rows):
        errors = ""
        valueList = tuple(rows.values())
        row = [valueList]

        try:
            errors = self.bigquery_client.insert_rows(self.table, row)
        except Exception as e:
            print(e)
 
        if len(errors) > 0:
            print(errors)
 
 
def weatherDataWriter(data):

    values = data
    bq_connector = BigQueryConnector(dataset_id=dataset_id, table_name=table_name_current, schema=currentSchema_raw)

    for value in values:
        bq_connector.writeRowsToStorage(value)
    print("Done with data handling...")
 
 
class WeatherFetcher:
    def __init__(self, api_key, towns):
        self.api_key = api_key
        self.towns = towns
        
    def getTimeOfDay(self):
        now = datetime.now().astimezone(timezone('Europe/Berlin'))
        hourOfDay_str = now.strftime("%Y-%m-%d %H:%M:%S").split(' ')[1].split(':')[0]
        timeOfDay = 'unknown'
        hourOfDay = float(hourOfDay_str)

        if(hourOfDay < 7): timeOfDay = 'night'
        if(hourOfDay >= 7 and hourOfDay < 11): timeOfDay = 'morning'
        if(hourOfDay >= 11 and hourOfDay < 14): timeOfDay = 'noon'
        if(hourOfDay >= 14 and hourOfDay < 18): timeOfDay = 'afternoon'
        if(hourOfDay >= 18 and hourOfDay < 22): timeOfDay = 'evening'
        if(hourOfDay >= 22): timeOfDay = 'night'

        return timeOfDay
 
    def getCurrentWeatherData(self):

        print("Fetching data by town")
        allWeatherData = []
        for town in self.towns:
            request_url = 'http://api.openweathermap.org/data/2.5/weather?q=' + town + '&units=metric&appid=' + self.api_key
            request_json = json.loads(requests.get(request_url).text)
 
            now = datetime.now().astimezone(timezone('Europe/Berlin'))
            timeNow = now.strftime("%Y-%m-%d %H:%M:%S")
            weekdayIndex = str(datetime.today().weekday()+1)
            weekdayLookup = { '1':'Monday', '2':'Tuesday','3':'Wednesday', '4':'Thursday', '5':'Friday', '6':'Saturday', '7':'Sunday' }

            rained = True if 'rain' in request_json else False;
            rainAmount_3hrs_mm = request_json['rain']['3h'] if 'rain' in request_json else 0;

            townWeather = {
                'date' : datetime.now().strftime("%Y-%m-%d"),
                'country' : request_json['sys']['country'],
                'town_name' : request_json['name'],
                'town_id' : str(request_json['id']),
                'currentTime' : timeNow,
                'timeZone' :  'Europe/Berlin',
                'timeOfDay' : self.getTimeOfDay(),
                'weekDay_Index' : weekdayIndex,
                'weekDay_Name' : weekdayLookup[weekdayIndex],
                'temperature' : request_json['main']['temp'],
                'overallWeather' : request_json['weather'][0]['main'],
                'overallWeather_code' : request_json['weather'][0]['id'],
                'rained' : rained,
                'rainAmount_3hrs_mm' : rainAmount_3hrs_mm,
                'humidity' : request_json['main']['humidity']
            }

            allWeatherData.append(townWeather)

        return allWeatherData

######

runWeatherDataSave()